In [1]:
import torch
import torchvision
from torchvision import transforms # 이미지 데이터 transform
from torch.utils.data import DataLoader # 이미지 데이터 로더
import os
import glob
import pandas as pd
import numpy as np

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [3]:
image_dir = './'
train_path = './train'
print(os.getcwd())
print(os.listdir(train_path))
num_classes = len(os.listdir(train_path))

c:\code\test_git\ResNet
['test_images_1', 'test_images_2', 'test_images_3', 'test_images_4', 'train_images_1', 'train_images_2', 'train_images_3']


In [4]:
import numpy as np

In [5]:
transform_train = transforms.Compose([
    transforms.Resize((256, 256)), # 이미지 resize
    transforms.RandomCrop(224), # 이미지를 랜덤으로 크롭
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2), # 이미지 지터링(밝기, 대조, 채비, 색조)
    transforms.RandomHorizontalFlip(p = 0.5), # p확률로 이미지 좌우반전
    transforms.RandomVerticalFlip(p = 0.5), # p확률로 상하반전
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

trainset = torchvision.datasets.ImageFolder(root=train_path, transform=transform_train)

In [6]:
import numpy as np

In [7]:
trainloader = DataLoader(trainset, batch_size=16,
                            shuffle=True, num_workers=3)

In [8]:
# import resnet
import torchvision.models.resnet as resnet
import torch.nn as nn
import torch.optim as optim

# 미리 정의
conv1x1=resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock= resnet.BasicBlock

In [9]:
class ResNet(nn.Module):
    
    def __init__(self, block, layers, num_classes, zero_init_residual=True):
        super(ResNet, self).__init__()
        self.inplanes = 64 

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False) 
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1) # 3 반복
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2) # 4 반복
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2) # 6 반복
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2) # 3 반복
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(2048, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1): # planes -> 입력되는 채널 수
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion: 
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [10]:
resnet50 = ResNet(resnet.Bottleneck, [3, 4, 6, 3], num_classes, True).to(device) 
# resnet50
# 1(conv1) + 9(layer1) + 12(layer2) + 18(layer3) + 9(layer4) +1(fc)= ResNet50

In [11]:
class Config:
  def __init__(self, **kwargs):
    for key, value in kwargs.items():
      setattr(self, key, value)

In [12]:
lr = 0.003
optimizer = 'Adam'

In [13]:
config = Config(
    trainloader = trainloader,
    model = resnet50,
    device = device,
    optimizer = torch.optim.Adam(resnet50.parameters(), lr=lr),
    criterion= nn.CrossEntropyLoss().to(device),
    globaliter = 0
)

In [14]:
class train_test():
      def __init__(self, config):
        # 파라미터 인자
        self.trainloader = config.trainloader
        self.model = config.model
        self.device = config.device
        self.optimizer = config.optimizer
        self.criterion = config.criterion
        self.globaliter = config.globaliter
        print(len(trainloader))
      def train(self, epochs, log_interval):
          self.model.train()
          for epoch in range(1, epochs + 1 ):  # epochs 루프
              running_loss = 0.0
              for i, data in enumerate(self.trainloader, 0): # batch 루프
                  # get the inputs
                  self.globaliter += 1
                  inputs, labels = data # input data, label 분리
                  inputs = inputs.to(self.device)
                  labels = labels.to(self.device)

                  # 가중치 초기화 -> 이전 batch에서 계산되었던 가중치를 0으로 만들고 최적화 진행
                  self.optimizer.zero_grad() 

                  # forward + backward + optimize
                  trainoutputs = self.model(inputs)
                  loss = self.criterion(trainoutputs, labels)
                  loss.backward()
                  self.optimizer.step()
                  lr_sche.step()
                  running_loss += loss.item()

              if epoch % log_interval == 0 :
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tlearningLoss: {:f}\twhole_loss: {:f} '.format(
                        epoch, i*len(inputs), len(self.trainloader.dataset),
                        100. * i*len(inputs) / len(self.trainloader.dataset), 
                        running_loss / log_interval,
                        loss.item()))
                running_loss = 0.0
      print('Finished Training')

Finished Training


In [15]:
ready_to_train=train_test(config)

33


In [16]:
lr_sche = optim.lr_scheduler.StepLR(config.optimizer, step_size=1000, gamma=0.2)
epochs = 300
log_interval = 10

ready_to_train.train(epochs, log_interval)

Train Epoch: 10 [416/525 (79%)]	learningLoss: 1.996840	whole_loss: 0.392375 
Train Epoch: 20 [416/525 (79%)]	learningLoss: 1.025900	whole_loss: 0.232520 
Train Epoch: 30 [416/525 (79%)]	learningLoss: 0.749429	whole_loss: 0.252382 
Train Epoch: 40 [416/525 (79%)]	learningLoss: 0.274407	whole_loss: 0.043722 
Train Epoch: 50 [416/525 (79%)]	learningLoss: 0.257460	whole_loss: 0.005099 
Train Epoch: 60 [416/525 (79%)]	learningLoss: 0.187436	whole_loss: 0.038794 
Train Epoch: 70 [416/525 (79%)]	learningLoss: 0.212457	whole_loss: 0.070533 
Train Epoch: 80 [416/525 (79%)]	learningLoss: 0.104051	whole_loss: 0.034140 
Train Epoch: 90 [416/525 (79%)]	learningLoss: 0.152569	whole_loss: 0.024478 
Train Epoch: 100 [416/525 (79%)]	learningLoss: 0.125111	whole_loss: 0.004499 
Train Epoch: 110 [416/525 (79%)]	learningLoss: 0.131163	whole_loss: 0.075035 
Train Epoch: 120 [416/525 (79%)]	learningLoss: 0.130535	whole_loss: 0.074009 
Train Epoch: 130 [416/525 (79%)]	learningLoss: 0.112648	whole_loss: 0.017

In [17]:
torch.save(ready_to_train.model,'resnet50.pt')